In [ ]:
#importing necessary libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs
from sklearn import linear_model
import statsmodels.api as sm 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# uploading data into the environment
main_df = pd.read_excel('/content/assignment_3_data.xlsx')
#acquiring original shape of the data to see how filtrtion changes it as well as for further error checking
shape_prior_to_filtrtation = main_df.shape
#looking at the uploaded data table
main_df.head(3)

,Global Company Key,Data Date,Data Year - Fiscal,Industry Format,Level of Consolidation - Company Annual Descriptor,Population Source,Data Format,Ticker Symbol,Company Name,ISO Currency Code,Assets - Total,Common Shares Outstanding,Debt in Current Liabilities - Total,Long-Term Debt - Total,Operating Income Before Depreciation,"Property, Plant and Equipment - Total (Net)",Stock Exchange Code,Active/Inactive Status Marker,Current ISO Country Code - Incorporation,Price Close - Annual - Fiscal,Standard Industry Classification Code
0,1036,1994-12-31,1994,INDL,C,D,STD,UDI.,UNITED DOMINION INDUSTRIES,USD,1359.356,39.443,9.234,309.768,166.234,222.148,11,I,CAN,19.125,3443
1,1081,1994-12-31,1994,INDL,C,D,STD,ABY.2,ABITIBI CONSOLIDATED INC,CAD,1821.000,87.415,44.000,479.000,107.000,906.000,11,I,CAN,19.375,2621
2,1238,1994-12-31,1994,INDL,C,D,STD,AOG.3,ALBERTA ENERGY CO LTD,CAD,2357.600,74.464,34.200,561.800,397.800,2017.700,11,I,CAN,17.875,1311


In [ ]:
filter_rule_1 = main_df[(main_df['Standard Industry Classification Code'] >=0000) & (main_df['Standard Industry Classification Code']<=999)].index

filter_rule_2 = main_df[(main_df['Standard Industry Classification Code'] >=4900) & (main_df['Standard Industry Classification Code']<=4999)].index

filter_rule_3 = main_df[(main_df['Standard Industry Classification Code'] >=6000) & (main_df['Standard Industry Classification Code']<=6999)].index

filter_rule_4 = main_df[main_df['Standard Industry Classification Code'] ==8888].index

filter_rule_5 = main_df[(main_df['Standard Industry Classification Code'] >=9000) & (main_df['Standard Industry Classification Code']<=9999)].index

#combining the first 5 filter rules to get a pandas index object that can be further used to delete unwanted observations
combined_filter_rules = filter_rule_1.union(filter_rule_2).union(filter_rule_3).union(filter_rule_4).union(filter_rule_5)

# applying the first 5 rules to filter the data frame. 
semi_filtered_df = main_df.drop(combined_filter_rules, axis=0)

# applying the last filter rule that is to drop missing values from stock market data (share price and shares outstanding)
filtered_df = semi_filtered_df.dropna(how = 'any', subset = ['Price Close - Annual - Fiscal', 'Common Shares Outstanding' ])




In [ ]:
#final_df = final_df.dropna()
filtered_df = filtered_df.dropna()


In [ ]:
# I will keep only those columns that are relevant for furhter econometric analysis: 
# Company Name, Ticker Symbol, Assets - Total, Common Shares Outstanding, Debt in Current Liabilities - Total, Long-Term Debt - Total, 
# Operating Income Before Depreciation, Property, Plant and Equipment - Total (Net), Price Close - Annual - Fiscal

final_df = filtered_df[['Company Name', 'Ticker Symbol', 'Assets - Total', 'Common Shares Outstanding', 'Debt in Current Liabilities - Total', 
             'Long-Term Debt - Total', 'Operating Income Before Depreciation', 'Property, Plant and Equipment - Total (Net)', 
             'Price Close - Annual - Fiscal' ]]
final_df.head()

,Company Name,Ticker Symbol,Assets - Total,Common Shares Outstanding,Debt in Current Liabilities - Total,Long-Term Debt - Total,Operating Income Before Depreciation,"Property, Plant and Equipment - Total (Net)",Price Close - Annual - Fiscal
0,UNITED DOMINION INDUSTRIES,UDI.,1359.356,39.443,9.234,309.768,166.234,222.148,19.125
1,ABITIBI CONSOLIDATED INC,ABY.2,1821.000,87.415,44.000,479.000,107.000,906.000,19.375
2,ALBERTA ENERGY CO LTD,AOG.3,2357.600,74.464,34.200,561.800,397.800,2017.700,17.875
3,ALCAN INC,AL.1,9989.000,224.685,265.000,2206.000,876.000,5534.000,25.375
5,BARRICK GOLD CORP,GOLD,3472.000,353.300,130.000,284.700,434.900,2768.300,22.250


## Part 1: 
1. Compute the variables in part (i). Generate summary statistics (mean, standard deviation,min and max) for book leverage, market leverage, profitability, tangibility, firm size, and
market-to-book.

In [ ]:
# Part 1: Compute the variables in part (i)
# Adding Total debt = dlc (Debt in Current Liabilities) + dltt (Long-Term Debt)
final_df['TotalDebt'] = final_df['Debt in Current Liabilities - Total'] + final_df['Long-Term Debt - Total']
# Book leverage = Total debt/ at (total assets)
final_df['BookLeverage'] = final_df['TotalDebt']/final_df['Assets - Total']
# Market leverage = Total debt/ (total debt + (prcc_f*csho))
final_df['MarketLeverage'] = final_df['TotalDebt']/(final_df['TotalDebt'] + 
                                                    (final_df['Price Close - Annual - Fiscal'] * final_df['Common Shares Outstanding']))
# Profitability = oibdp / at
final_df['profitability'] = final_df['Operating Income Before Depreciation']/final_df['Assets - Total']
# Tangibility = ppent / at
final_df['tangibility'] = final_df['Property, Plant and Equipment - Total (Net)']/final_df['Assets - Total']
# Firm size = log(at)
final_df['firm_size'] = np.log(final_df['Assets - Total'])
# MB (market-to-book) = (prcc_f*csho+dlc+dltt)/at
final_df['mb'] = ((final_df['Price Close - Annual - Fiscal'] * final_df['Common Shares Outstanding']) + final_df['TotalDebt'])/final_df['Assets - Total']




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [ ]:
final_df.head(2)

,Company Name,Ticker Symbol,Assets - Total,Common Shares Outstanding,Debt in Current Liabilities - Total,Long-Term Debt - Total,Operating Income Before Depreciation,"Property, Plant and Equipment - Total (Net)",Price Close - Annual - Fiscal,TotalDebt,BookLeverage,MarketLeverage,profitability,tangibility,firm_size,mb
0,UNITED DOMINION INDUSTRIES,UDI.,1359.356,39.443,9.234,309.768,166.234,222.148,19.125,319.002,0.234671,0.297202,0.122289,0.163422,7.214766,0.789601
1,ABITIBI CONSOLIDATED INC,ABY.2,1821.000,87.415,44.000,479.000,107.000,906.000,19.375,523.000,0.287205,0.235940,0.058759,0.497529,7.507141,1.217279


In [ ]:
# Generate summary statistics (mean, standard deviation, min and max) for book leverage, market leverage,
# profitability, tangibility, firm size, and market-to-book.
print(final_df['BookLeverage'].describe())

count    83.000000
mean      0.311483
std       0.166331
min       0.001560
25%       0.224619
50%       0.283713
75%       0.409125
max       0.909118
Name: BookLeverage, dtype: float64


In [ ]:
print(final_df['MarketLeverage'].describe())

count    83.000000
mean      0.350140
std       0.198652
min       0.001748
25%       0.200872
50%       0.345419
75%       0.476910
max       0.927795
Name: MarketLeverage, dtype: float64


In [ ]:
# Calculating summary statistics for profitability:
print(final_df['profitability'].describe())

count    83.000000
mean      0.126450
std       0.055358
min      -0.003353
25%       0.090700
50%       0.121448
75%       0.156826
max       0.362329
Name: profitability, dtype: float64


In [ ]:
# Calculating summary statistics for tangibility:
print(final_df['tangibility'].describe())

count    83.000000
mean      0.540139
std       0.212742
min       0.064432
25%       0.401695
50%       0.561965
75%       0.695916
max       0.957422
Name: tangibility, dtype: float64


In [ ]:
# Calculating summary statistics for firm_size:
print(final_df['firm_size'].describe())

count    83.000000
mean      7.835645
std       0.711808
min       6.917630
25%       7.300183
50%       7.728114
75%       8.156599
max      10.547760
Name: firm_size, dtype: float64


In [ ]:
# Calculating summary statistics for mb:
print(final_df['mb'].describe())

count    83.000000
mean      0.999150
std       0.503719
min       0.279159
25%       0.724091
50%       0.895368
75%       1.072869
max       4.085201
Name: mb, dtype: float64


## Part 2: Using book leverage (model 1) and market leverage (model 2), estimate the following models of corporate leverage:
𝐿𝑒𝑣𝑒𝑟𝑎𝑔𝑒=𝛽0+𝛽1𝑃𝑟𝑜𝑓𝑖𝑡𝑎𝑏𝑖𝑙𝑖𝑡𝑦+𝛽2𝑇𝑎𝑛𝑔𝑖𝑏𝑖𝑙𝑖+𝛽3𝐹𝑖𝑟𝑚𝑠𝑖𝑧𝑒+𝛽4𝑀𝐵+𝑢

In [ ]:
# preparing data to input it into the modeL:
X = final_df[['profitability', 'tangibility','firm_size', 'mb']] 
# Dependent variable for model 1:
book_lev = final_df['BookLeverage']
# Dependent variable for model 2:
mrkt_lev = final_df['MarketLeverage']
# adding constant for the regression analysis
X = sm.add_constant(X)
# Creating model object
#model = sm.OLS(y,X)
#results = model.fit()
#results.summary()

In [ ]:
# Model 1 (book leverage): 
model_1 = sm.OLS(book_lev, X)
results_1 = model_1.fit()
results_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           BookLeverage   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     1.460
Date:                Sun, 28 Mar 2021   Prob (F-statistic):              0.223
Time:                        10:18:42   Log-Likelihood:                 34.610
No. Observations:                  83   AIC:                            -59.22
Df Residuals:                      78   BIC:                            -47.13
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.2229      0.216      1.031      0.306      -0.207       0.653
profitability    -0.5923      0.370     -1.599      0.114      -1.330       0.145
tangibility      -0.0071      0.092     -0.077      0.938      -0.191       0.177
firm_size         0.0103      0.026      0.393      0.695      -0.042       0.063
mb                0.0866      0.040      2.185      0.032       0.008       0.166
==============================================================================
Omnibus:                        0.368   Durbin-Watson:                   2.190
Prob(Omnibus):                  0.832   Jarque-Bera (JB):                0.441
Skew:                           0.151   Prob(JB):                        0.802
Kurtosis:                       2.810   Cond. No.                         167.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Model 2 (market leverage):
model_2 = sm.OLS(mrkt_lev, X)
results_2 = model_2.fit()
results_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         MarketLeverage   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     4.579
Date:                Sun, 28 Mar 2021   Prob (F-statistic):            0.00225
Time:                        12:50:42   Log-Likelihood:                 25.629
No. Observations:                  83   AIC:                            -41.26
Df Residuals:                      78   BIC:                            -29.16
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.5156      0.241      2.141      0.035       0.036       0.995
profitability    -0.7197      0.413     -1.744      0.085      -1.541       0.102
tangibility      -0.0294      0.103     -0.286      0.775      -0.234       0.175
firm_size         0.0076      0.029      0.261      0.795      -0.051       0.066
mb               -0.1184      0.044     -2.680      0.009      -0.206      -0.030
==============================================================================
Omnibus:                        0.560   Durbin-Watson:                   2.150
Prob(Omnibus):                  0.756   Jarque-Bera (JB):                0.694
Skew:                           0.166   Prob(JB):                        0.707
Kurtosis:                       2.699   Cond. No.                         167.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Part 4: Estimate regression equation (1) for book and market leverage using robust standard errors.

In [ ]:
# Model 1 robust standard errors (book leverage): 
model_robust_1 = sm.OLS(book_lev, X)
results_robust_1 = model_robust_1.fit()
results_robust_1.get_robustcov_results(cov_type='HC3').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           BookLeverage   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                    0.6712
Date:                Sun, 28 Mar 2021   Prob (F-statistic):              0.614
Time:                        13:09:58   Log-Likelihood:                 34.610
No. Observations:                  83   AIC:                            -59.22
Df Residuals:                      78   BIC:                            -47.13
Df Model:                           4                                         
Covariance Type:                  HC3                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.2229      0.200      1.115      0.268      -0.175       0.621
profitability    -0.5923      0.421     -1.408      0.163      -1.430       0.245
tangibility      -0.0071      0.086     -0.083      0.934      -0.178       0.164
firm_size         0.0103      0.021      0.481      0.632      -0.032       0.053
mb                0.0866      0.126      0.685      0.495      -0.165       0.338
==============================================================================
Omnibus:                        0.368   Durbin-Watson:                   2.190
Prob(Omnibus):                  0.832   Jarque-Bera (JB):                0.441
Skew:                           0.151   Prob(JB):                        0.802
Kurtosis:                       2.810   Cond. No.                         167.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [ ]:
# Model 2 robust standard errors  (book leverage): 
model_robust_2 = sm.OLS(mrkt_lev, X)
results_robust_2 = model_robust_2.fit()
results_robust_2.get_robustcov_results(cov_type='HC3').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         MarketLeverage   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     1.707
Date:                Sun, 28 Mar 2021   Prob (F-statistic):              0.157
Time:                        13:23:44   Log-Likelihood:                 25.629
No. Observations:                  83   AIC:                            -41.26
Df Residuals:                      78   BIC:                            -29.16
Df Model:                           4                                         
Covariance Type:                  HC3                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.5156      0.246      2.094      0.040       0.025       1.006
profitability    -0.7197      0.440     -1.636      0.106      -1.596       0.156
tangibility      -0.0294      0.095     -0.310      0.758      -0.219       0.160
firm_size         0.0076      0.026      0.295      0.769      -0.044       0.059
mb               -0.1184      0.102     -1.161      0.249      -0.321       0.085
==============================================================================
Omnibus:                        0.560   Durbin-Watson:                   2.150
Prob(Omnibus):                  0.756   Jarque-Bera (JB):                0.694
Skew:                           0.166   Prob(JB):                        0.707
Kurtosis:                       2.699   Cond. No.                         167.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""